In [ ]:
# A bit of setup
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Create color maps
cmap_list = ['orange', 'cyan', 'cornflowerblue']
cmap_bold = ['darkorange', 'c', 'darkblue']
cmap_light = ListedColormap(cmap_list)

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Let's make some data

We want a problem that is not linearly separable to show how poorly a linear classification would work here.

Note later that centering at zero is helpful for us.  We will make three classes, but you could add more or less if you wish by adjusting the variable K.  Feel free to add more dimensions, but it's harder to view beyond 2.

In [ ]:
np.random.seed(10)

N = 150 # number of points per class
D = 2 # dimensionality
K = 3 # number of classes

X = np.zeros((N*K,D))
y = np.zeros(N*K, dtype='uint8')

easy = 0  #set this to 1/True to have an easier data sample to train on
slopeDiff = 0.0
offsetDiff = 0.3

min = -1
max = 1
if easy:
    min = -0.5
    max = 2
    
for j in range(K):
    ix = range(N*j,N*(j+1))
    r = np.linspace(0.0,1,N) # radius
    t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2 # theta
    xx = r*np.sin(t)
    yy = r*np.cos(t)
    if easy:
        xx = np.linspace(0,1,N)
        yy = (0.5+0.5*slopeDiff*(j+1))*xx + np.random.randn(N)*(0.05+j*0.05) + j*offsetDiff
    
    X[ix] = np.c_[xx,yy]
    y[ix] = j

fig = plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim([min,max])
plt.ylim([min,max])
#fig.savefig('spiral_raw.png')

# Train a neural network with one hidden layer

The linear classifier approach was fine, but it didn't correctly classify in more than ~50% of cases.  So we can ramp this up to a neural network.  We are still going to make linear functions of our input variables, $f(X) = W^TX+b$, but now we will use activation functions to introduce a non-linearity.  In this example we'll use "Rectified Linear Units", aka ReLU, as our activation function.  

This NN will have one input layer (2 inputs), one hidden layer (100 neurons by default) and one output node that receives all of the activation function outputs. In the previous example we had $2\times 3 + 3 =9$ weights.  Now we will have $2\times 100 + 100=300$ weights for the inputs to the hidden layer and $100 \times 3 + 3 = 303$ weights for the output node.  So we had better be able to improve!

But we need to study some activation functions first.  Let's see what we can find out about the following options:
  * Rectified Logical Unit (ReLU)
  * Sigmoid
  * SoftPlus
  * Leaky ReLU
  * Exponential ReLU (ELU)

In [ ]:
# define our NN activation function here
def activation(i, input):

    if i == 0:
        # Rectified linear unit (ReLU) activation
        output = np.maximum(0,input)
    elif i == 1:
        #Sigmoid
        output =  1/(1+np.exp(-1*input))
    elif i == 2:
        # SoftPlus
        output = np.log(1+np.exp(input))
    elif i == 3:
        # Leaky ReLU
        output = np.maximum(0.1*input,0.9*input)
    elif i == 4:
        # Exponential ReLU
        output = np.where(input>0,input,np.exp(input)-1)
    else:
        return input
    
    return output

ix = np.linspace(-4,4,200)

plt.plot(ix,activation(0,ix),color="red",label="ReLU")
plt.plot(ix,activation(1,ix),color="blue",label="Sigmoid")
plt.plot(ix,activation(2,ix),color="green",label="SoftPlus")
plt.plot(ix,activation(3,ix),color="magenta",label="Leaky ReLU")
plt.plot(ix,activation(4,ix),color="orange",label="ELU")

plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# initialize parameters randomly
h = 100 # size of hidden layer
W = 0.01 * np.random.randn(D,h)
b = np.zeros((1,h))

W2 = 0.01 * np.random.randn(h,K)
b2 = np.zeros((1,K))

# some hyperparameters
step_size = 1
reg = 1e-3 # regularization strength

iActivate = 0

# gradient descent loop
num_examples = X.shape[0]
for i in range(10001):
  
    # evaluate class scores, [N x K]
    # note that fhat is the same as before: fhat=W^T X + b
    fhat = np.dot(X, W) + b
    
    # but now we pass the linear sum into an activation function
    # this provides a nonlinearity that wasn't there for the previous case
    hidden_layer = activation(iActivate,fhat)
    
    # this is the fully-connected output layer, just summing the output
    # of the activation functions
    scores = np.dot(hidden_layer, W2) + b2
  
    # compute the class probabilities
    # this is softmax, converting to class probability
    # prob1 = exp(score1)/{exp(score1)+exp(score2)+...}
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]
  
    # compute the loss: average cross-entropy loss and regularization
    corect_logprobs = -np.log(probs[range(num_examples),y])
    data_loss = np.sum(corect_logprobs)/num_examples
    
    # this time we have two weight matrices, so we need to include
    # both in the L2 regularization
    reg_loss = 0.5*reg*np.sum(W*W) + 0.5*reg*np.sum(W2*W2)
    loss = data_loss + reg_loss
  
    if i % 1000 == 0:
        print("iteration %d: loss %f" % (i, loss))
  
    # compute the gradient on scores
    dscores = probs
    dscores[range(num_examples),y] -= 1
    dscores /= num_examples
  
    # backpropate the gradient to the parameters
    # first backprop into parameters W2 and b2
    dW2 = np.dot(hidden_layer.T, dscores)
    db2 = np.sum(dscores, axis=0, keepdims=True)

    # next backprop into hidden layer
    dhidden = np.dot(dscores, W2.T)
    # backprop the ReLU non-linearity
    dhidden[hidden_layer <= 0] = 0
    
    # finally into W,b
    dW = np.dot(X.T, dhidden)
    db = np.sum(dhidden, axis=0, keepdims=True)
  
    # add regularization gradient contribution
    dW2 += reg * W2
    dW += reg * W
  
    # perform a parameter update
    W += -step_size * dW
    b += -step_size * db
    W2 += -step_size * dW2
    b2 += -step_size * db2

In [ ]:
# evaluate training set accuracy
hidden_layer = activation(iActivate,fhat)
scores = np.dot(hidden_layer, W2) + b2
predicted_class = np.argmax(scores, axis=1)
print('training accuracy: %.2f' % (np.mean(predicted_class == y)))

In [ ]:
# plot the resulting classifier
h = 0.02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
iArg = np.dot(np.c_[xx.ravel(), yy.ravel()], W) + b
Z = np.dot(activation(iActivate,iArg), W2) + b2

Z = np.argmax(Z, axis=1)
Z = Z.reshape(xx.shape)

fig = plt.figure()
plt.contourf(xx, yy, Z, cmap=cmap_light, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
#fig.savefig('spiral_net.png')

In [ ]:
# We have N x 3 and need the first column split by 
# If we pick class = 2, that gives us "blue vs the rest"
iClass = 2
Xplt = np.zeros((N,3))
Xplt[:,0] = scores[:N,iClass]
Xplt[:,1] = scores[N:2*N,iClass]
Xplt[:,2] = scores[2*N:3*N,iClass]

plt.hist(Xplt,30,color=cmap_list)
plt.show()